In [ ]:
# Login to Hugging Face Hub to access models and datasets.
from huggingface_hub import login
login()

In [ ]:
# Install the required libraries for the project.
!pip install -r requirements-vi.txt

  Cloning https://github.com/kpu/kenlm.git to /tmp/pip-req-build-sgn8kfw0
  Running command git clone --filter=blob:none --quiet https://github.com/kpu/kenlm.git /tmp/pip-req-build-sgn8kfw0
  Resolved https://github.com/kpu/kenlm.git to commit 4cb443e60b7bf2c0ddf3c745378f76cb59e254e5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [ ]:
# Load the Common Voice dataset for Vietnamese.
from datasets import load_dataset, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "vi", split="train+validation")
common_voice_test = load_dataset("mozilla-foundation/common_voice_17_0", "vi", split="test")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


README.md:   0%|          | 0.00/12.7k [00:00<?, ?B/s]

common_voice_17_0.py:   0%|          | 0.00/8.19k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/132k [00:00<?, ?B/s]

...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 11533it [00:00, 63484.17it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 377it [00:00, 56493.48it/s]


Generating validated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 5135it [00:00, 42672.33it/s]


In [ ]:
# Remove unnecessary columns from the dataset.
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
# Function to display random samples from the dataset.
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence,variant
0,Bởi đối với cô,
1,Được thôi mấy cô cậu cứ tự nhiên,
2,Trên bầu trời cao,
3,sự sám hối,
4,Tổng hợp những bài thơ hay nhất,
5,Tai vách mạch rừng.,
6,Tội nghiệp con Thảo đã chết mà còn khổ,
7,Tình bạn chúng ta,
8,Đám tang cũng đã lo xong rồi,
9,nó gắn liền với mạng sống và tương lai của cô,


In [ ]:
# Function to remove special characters from the sentences.
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\»\«]'

def remove_special_characters(batch):
    # remove special characters
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()

    return batch

common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

Map:   0%|          | 0/2939 [00:00<?, ? examples/s]

Map:   0%|          | 0/1274 [00:00<?, ? examples/s]

,sentence,variant
0,anh điện cũng chỉ được thắp sáng trong từng ngôi nhà,
1,nghe mẹ thảo nói bố thảo đi làm trưa mới về,
2,bực thế chứ xe để đây sao lại xịt lốp được,
3,bên trong nhà bỗng vang lên một giọng nói ma mị,
4,em mặc như vậy không có sợ lạnh sao,
5,alo anh quân à,
6,tất cả,
7,mẹ mừng con trúng,
8,bà nói cho chúng tôi biết lý do được không,
9,anh anh biến đi đâu rồi mà lại xuất hiện như vậy,


In [ ]:
# Extract all unique characters from the dataset to create a vocabulary.
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

Map:   0%|          | 0/2939 [00:00<?, ? examples/s]

Map:   0%|          | 0/1274 [00:00<?, ? examples/s]

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 'à': 27,
 'á': 28,
 'â': 29,
 'ã': 30,
 'è': 31,
 'é': 32,
 'ê': 33,
 'ì': 34,
 'í': 35,
 'ò': 36,
 'ó': 37,
 'ô': 38,
 'õ': 39,
 'ù': 40,
 'ú': 41,
 'ý': 42,
 'ă': 43,
 'đ': 44,
 'ĩ': 45,
 'ũ': 46,
 'ơ': 47,
 'ư': 48,
 'ạ': 49,
 'ả': 50,
 'ấ': 51,
 'ầ': 52,
 'ẩ': 53,
 'ẫ': 54,
 'ậ': 55,
 'ắ': 56,
 'ằ': 57,
 'ẳ': 58,
 'ẵ': 59,
 'ặ': 60,
 'ẹ': 61,
 'ẻ': 62,
 'ẽ': 63,
 'ế': 64,
 'ề': 65,
 'ể': 66,
 'ễ': 67,
 'ệ': 68,
 'ỉ': 69,
 'ị': 70,
 'ọ': 71,
 'ỏ': 72,
 'ố': 73,
 'ồ': 74,
 'ổ': 75,
 'ỗ': 76,
 'ộ': 77,
 'ớ': 78,
 'ờ': 79,
 'ở': 80,
 'ỡ': 81,
 'ợ': 82,
 'ụ': 83,
 'ủ': 84,
 'ứ': 85,
 'ừ': 86,
 'ử': 87,
 'ữ': 88,
 'ự': 89,
 'ỳ': 90,
 'ỵ': 91,
 'ỷ': 92,
 'ỹ': 93}

In [ ]:
# Add special tokens to the vocabulary.
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

96

In [ ]:
# Create a tokenizer and a feature extractor, then combine them into a processor.
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

from huggingface_hub import hf_hub_download
from importlib.machinery import SourceFileLoader
from transformers import Wav2Vec2Processor

MODEL_ID = "nguyenvulebinh/wav2vec2-base-vi-vlsp2020"

# download the model_handling.py helper
model_script = hf_hub_download(repo_id=MODEL_ID, filename="model_handling.py")

# load the class that wraps the CTC model
model_loader = SourceFileLoader("model_handling", model_script).load_module()

# instantiate processor + model
processor = Wav2Vec2Processor.from_pretrained("nguyenvulebinh/wav2vec2-base-vi-vlsp2020")

model     = model_loader.Wav2Vec2ForCTC.from_pretrained(MODEL_ID)
common_voice_train[0]["audio"]

model_handling.py: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/385M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/385M [00:00<?, ?B/s]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/988a1303a5bdc3cbd1fbdc27b2aeeb0c081ca80a35035ee932f592ffd443f607/vi_train_0/common_voice_vi_23901117.mp3',
 'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.90132641e-06, 7.18296724e-06, 1.22913943e-05]),
 'sampling_rate': 48000}

In [ ]:
# Resample the audio to 16kHz.
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/988a1303a5bdc3cbd1fbdc27b2aeeb0c081ca80a35035ee932f592ffd443f607/vi_train_0/common_voice_vi_23901117.mp3',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.79643739e-05, -1.54107620e-05,  3.32880700e-06]),
 'sampling_rate': 16000}

In [ ]:
# Display a random audio sample and its transcription.
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

thì sẽ bị thảo dắt đi cùng


In [ ]:
# Print information about a random audio sample.
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: khổthân cái hạ thật đấy
Input array shape: (50688,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    array = audio["array"]
    sr    = audio["sampling_rate"]
    pe = processor(array, sampling_rate=sr)
    input_vals = pe.input_values[0]

    batch["input_values"] = input_vals

    batch["input_length"]   = len(input_vals)

    labels = processor(text=batch["sentence"]).input_ids
    batch["labels"] = labels

    return batch

# Remap your datasets
common_voice_train = common_voice_train.map(
    prepare_dataset,
    remove_columns=common_voice_train.column_names,
    num_proc=4,               # speed up if you have CPUs
    batched=False             # since we index [0], keep it per-example
)
common_voice_test = common_voice_test.map(
    prepare_dataset,
    remove_columns=common_voice_test.column_names,
    num_proc=4,
    batched=False
)


Map (num_proc=4):   0%|          | 0/2939 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1274 [00:00<?, ? examples/s]

In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Union
import torch
from transformers import Wav2Vec2Processor

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # features now have "input_values" and "labels"
        input_features = [{"input_values": feat["input_values"]} for feat in features]
        label_features = [{"input_ids": feat["labels"]} for feat in features]

        # pad inputs
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # pad labels
        labels_batch = self.processor.pad(
            labels=label_features,
            padding=self.padding,
            return_tensors="pt",
        )
        # replace padding token id's with -100 so they are ignored by CTC loss
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels

        return batch

# now instantiate with Wav2Vec2Processor
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [ ]:
# Define a class to compute the Word Error Rate (WER) metric.
import re
from typing import List, Dict

class WordErrorRate:
    """Low‑level WER calculator (as before)."""
    def __init__(self):
        self._punct = re.compile(r"[^\w\s]")

    def _normalize(self, text: str) -> List[str]:
        text = self._punct.sub("", text.lower()).strip()
        return text.split()

    def _levenshtein(self, ref: List[str], hyp: List[str]) -> Dict[str, int]:
        n, m = len(ref), len(hyp)
        dp = [[0]*(m+1) for _ in range(n+1)]
        for i in range(1,n+1): dp[i][0]=i
        for j in range(1,m+1): dp[0][j]=j

        for i in range(1,n+1):
            for j in range(1,m+1):
                dp[i][j] = (
                    dp[i-1][j-1]
                    if ref[i-1]==hyp[j-1]
                    else 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
                )

        i, j = n, m
        subs = dels = ins = 0
        while i>0 or j>0:
            if i>0 and j>0 and ref[i-1]==hyp[j-1]:
                i, j = i-1, j-1
            else:
                opts = {}
                if i>0 and j>0: opts['sub']=dp[i-1][j-1]
                if i>0:          opts['del']=dp[i-1][j]
                if j>0:          opts['ins']=dp[i][j-1]
                op = min(opts, key=opts.get)
                if op=='sub':
                    subs += 1; i, j = i-1, j-1
                elif op=='del':
                    dels += 1; i -= 1
                else:
                    ins += 1; j -= 1
        return {"substitutions": subs, "deletions": dels, "insertions": ins}

    def compute(self, preds: List[str], refs: List[str]) -> float:
        assert len(preds)==len(refs), "preds/refs must align"
        err = ref_words = 0
        for p, r in zip(preds, refs):
            ptoks = self._normalize(p)
            rtoks = self._normalize(r)
            c = self._levenshtein(rtoks, ptoks)
            err += c["substitutions"] + c["deletions"] + c["insertions"]
            ref_words += len(rtoks)
        return err/ref_words if ref_words>0 else 0.0


class WERMetric:
    """
    Mimics 🤗datasets load_metric("wer"):
      - wer_metric.add(pred, ref) or add_batch([...], [...])
      - wer_metric.compute() → {'wer': float}
    """
    def __init__(self):
        self._wer = WordErrorRate()
        self._preds: List[str] = []
        self._refs:  List[str] = []

    def add(self, prediction: str, reference: str):
        self._preds.append(prediction)
        self._refs.append(reference)

    def add_batch(self, predictions: List[str], references: List[str]):
        if len(predictions) != len(references):
            raise ValueError("predictions and references must have same length")
        self._preds.extend(predictions)
        self._refs.extend(references)

    def compute(self) -> Dict[str, float]:
        score = self._wer.compute(self._preds, self._refs)
        self._preds.clear()
        self._refs.clear()
        return {"wer": score}


# Usage example:
wer_metric = WERMetric()
wer_metric.add_batch(
    ["hello world",    "this is a test"],
    ["hello word",     "this is test"]
)
result = wer_metric.compute()
print(result)  # → {'wer': 0.4}

{'wer': 0.4}


In [ ]:
# Define a function to compute the metrics during training.
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer_metric.add_batch(predictions=pred_str, references=label_str)
    wer = wer_metric.compute()

    return {"wer": wer}

In [ ]:
# Define the training arguments for the Trainer
from transformers import TrainingArguments

#These settings are used on hf blog on fine tuning STT
training_args = TrainingArguments(
  output_dir="vietnamese-model-test",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  eval_strategy="steps",
  num_train_epochs=10,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=600,
  eval_steps=300,
  logging_steps=300,
  learning_rate=5e-5,
  warmup_steps=500,
  save_total_limit=2
)
"""
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="vietnamese-model-test",
  # max_steps=1,
  num_train_epochs=10,
  # --- Memory Optimization Settings ---
  per_device_train_batch_size=1,      # 📉 Lower the batch size
  gradient_accumulation_steps=1,
  gradient_checkpointing=True,        # ✅ Re-enable for memory savings

  # --- Other Settings ---
  eval_strategy="steps",
  eval_steps=1000,
  logging_steps=1,
  fp16=True,
  learning_rate=5e-5,
  group_by_length=True,
  push_to_hub=False,
)
"""

from transformers import Trainer
# 1) Instantiate Trainer (you already have this)
t = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor,
)

# 2) Evaluate before training
eval_results = t.evaluate()
print("Before training →", eval_results)

# 3) (Optional) Predict on test set
preds, labels, test_metrics = t.predict(common_voice_test)
print("Predict metrics →", test_metrics)

PyTorch: setting up devices
average_tokens_across_devices is True but world size is 1. Setting it to False automatically.
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/tmp/ipython-input-21-1388273415.py:46: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  t = Trainer(
Using auto half precision backend
The following columns in the Evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1274
  Batch size = 8


Trainer is attempting to log a value of "{'wer': 0.17132570944515035}" of type <class 'dict'> for key "eval/wer" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ngoxuankien2005 (ngoxuankien2005-university-of-science-and-technology-of-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


The following columns in the test set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.


Before training → {'eval_loss': 0.32528460025787354, 'eval_model_preparation_time': 0.0053, 'eval_wer': {'wer': 0.17132570944515035}, 'eval_runtime': 53.2373, 'eval_samples_per_second': 23.931, 'eval_steps_per_second': 3.005}



***** Running Prediction *****
  Num examples = 1274
  Batch size = 8


Predict metrics → {'test_loss': 0.3250599205493927, 'test_model_preparation_time': 0.0053, 'test_wer': {'wer': 0.17090216010165185}, 'test_runtime': 51.4076, 'test_samples_per_second': 24.782, 'test_steps_per_second': 3.112}


In [ ]:
# Start the training process.
t.train()

The following columns in the Training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 2,939
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 920
  Number of trainable parameters = 96,223,458


Step,Training Loss,Validation Loss,Model Preparation Time,Wer
300,0.255500,0.282301,0.005300,{'wer': 0.16666666666666666}
600,0.172600,0.329275,0.005300,{'wer': 0.17947903430749682}


The following columns in the Evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1274
  Batch size = 8
Trainer is attempting to log a value of "{'wer': 0.16666666666666666}" of type <class 'dict'> for key "eval/wer" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
The following columns in the Evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1274
  Batch size = 8
Trainer is attempting to log a value of "{'wer': 0.17947903430749682}" of type <class 'dict'> for key "eval/wer" as a scalar. This invocation of Ten

Step,Training Loss,Validation Loss,Model Preparation Time,Wer
300,0.255500,0.282301,0.005300,{'wer': 0.16666666666666666}
600,0.172600,0.329275,0.005300,{'wer': 0.17947903430749682}
